## A look at Taylor Series Expansions of a simple Gaussian function

### Click on the "run" button above to run each cell in turn; wait for the \* in the square brackets to go away before continuing

### Background:

The following homework problem was assigned to my Physical Chemistry Laboratory students (last year):

5.  (**a**)  Find the first 4 non-zero terms in the MacLaurin Expansion of $\int_0^1 \rm{e}^{-x^2} dx$ .  
(**b**)  Estimate the value of the integral $\int_0^1 \rm{e}^{-x^2} dx$  by integrating the terms of the MacLaurin Expansion of the integrand using the first 2 terms, and separately using the first 4 terms (*i.e.* get two different estimates).
(**c**)  Estimate the value of the integral $\int_0^5 \rm{e}^{-x^2} dx$  by integrating the terms of the MacLaurin Expansion of the integrand using the first 2 terms, and separately using the first 4 terms (*i.e.* get two different estimates). 
(**d**)  Comment on the results of parts (**b**) and (c).  What, if anything, appears unusual?  Can you explain it? 


In [ ]:
# initialization stuff
from pylab import *
%matplotlib inline
import sympy as sym
from IPython.display import display, Latex
from scipy.special import factorial

# set up variable and Gaussian function
x = sym.symbols('x')
f = sym.exp(-x**2)

def int_result(f, lower, upper, var = x, enclose = False, showN = False):
    result = sym.integrate(f, var)
    result = sym.latex(result)
    #lupper = '\{ {} \}'.format(latex(upper))
    # hand-made replacement for log
    result = result.replace("\log","\ln")
    result = r"\left( {} \right) \bigg \rvert_{{ {} }}^{{ {} }}".format(result, sym.latex(lower), sym.latex(upper))
    
    final = sym.integrate(f, (var, lower, upper))
    Nfinal = sym.N(final)
    final = sym.latex(final.simplify())
    final = final.replace("\log", "\ln")
    int_format = "\int_{{ {} }}^{{  {}  }} ( {} ) d{}" if enclose else "\int_{{ {} }}^{{ {} }} {} d{}"
    integral = int_format.format(sym.latex(lower), sym.latex(upper), sym.latex(f), sym.latex(var))
    if showN:
        output = "$${0} = {1} = {2} = {3:.4f}$$".format(sym.latex(integral), result, final, Nfinal)
    else:
        output = "$${} = {} = {}$$".format(sym.latex(integral), result, final)
    display(Latex(output))
        
def display_int(fun, a, b,  inc = None):
    ''' show integral graphically:
        a, b    original integral interval
        inc     increment to add to either side
        fun     scipy function for integrand'''
    if not inc:
        inc = (b - a) / 20
    da = a - inc
    db = b + inc
    xrng = linspace(da, db, 200)
    yrng = fun(xrng)
    intrng = linspace(a, b, 200)
    plot(xrng, yrng, 'm', linewidth = 1.5)
    grid()
    fill_between(intrng, fun(intrng), hatch = '/', alpha = 0.5)

In [ ]:
# NOTE:  if you run the following code block, all of the code blocks,
# including this one, will be hidden, and you can see just the results
# of everythin.  To undo that, select cell -> All Output -> Clear from
# the menu bar just under the "jupyter" header above
# For best effect, run all of the code cells EXCEPT the 'html' one immediately
# below this, then come back and run the 'html' cell.

In [ ]:
%%html
<style>
div.input {
    display:none;
}
</style>

### Solutions:
 (**a**)  $f(x) = \rm{e}^{-x^2}$ is easy to expand in a Taylor series; just use the known Taylor series expansion for $\rm{e}^x$:
$$\rm{e}^x = 1 + x + \frac{x^2}{2} + \frac{x^3}{3!} + \frac{x^4}{4!} + ...$$
and substitute in $-x^2$ in place of the 'x' in this series to get:
$$\rm{e}^{-x^2} = 1 - x^2 + \frac{x^4}{2} - \frac{x^6}{3!} + \frac{x^8}{4!} - ...$$

Aside:  
Here is how to do this using the Python sympy package:

In [ ]:
# Yes, you can go out further than 32 terms; this gets the point across, though
for N in [4, 8, 16, 32]:
    result = "$${} = {}$$".format("f(x)", sym.latex(f.series(x,0,N)))
    display(Latex(result))

To calculate the expansion efficiently, define a function that sums up the terms up to (but not including) the Nth power:

In [ ]:
# define numerical functions to use below

from scipy import vectorize

@vectorize
def TayGauss(x, N = 4):
    # each term is of form (-1)^n x^(2n)/n!
    result = sum([(-x**2)**n / factorial(n) for n in range(N//2)])
    return result

# numerical Gaussian:
def gauss(x):
    return exp(-x**2)

Compare the results of the analytical expansion to that of the numerical calculation to be convincing:  Choose different values of N in the cell below, should get the same numerical results from both methods of expansion:

In [ ]:
# Choose number of terms by changing value of N in below line:
N = 8
# Set value of x at which to evaluate expansion:
xtry = 0.5

eval_f = f.series(x, 0, N).removeO()
analytical_result = eval_f.subs(x, xtry)
numerical_result  = TayGauss(xtry, N)
true_value = f.subs(x, xtry)
print(f"Results at x = {xtry}")
print(f"True Value:           {true_value}")
print(f"Analytical Expansion: {analytical_result}")
print(f"Numerical Expansion:  {numerical_result}")

(**b**)  Use 2-term and 4-term expansions to estimate value of $\int_0^1 \rm{e}^{-x^2} dx$.


In [ ]:
print("True Result")
int_result(f, 0, 1, showN=True)
for n_terms in [4, 8]:
    g = f.series(x, 0, n_terms).removeO()
    print("%d-term approximation" % (n_terms//2))
    int_result(g, 0, 1, enclose = True, showN=True)

These results seem reasonable; clearly, the 4-term approximation does a better job.  Nothing wildly surprising.

Let's look at the integrals in more detail here

In [ ]:
from scipy.integrate import quad
ans = quad(gauss, 0, 1)[0]
print(f"Integral of actual Gaussian {ans:.6}")
display_int(gauss, 0, 1)


In [ ]:
ans = quad(TayGauss, 0, 1)[0]
print(f"Integral of 2-term approximation {ans:.6}")
display_int(TayGauss, 0, 1)

In [ ]:

def taylor8(x):
    return TayGauss(x, N=8)
ans = quad(taylor8, 0, 1)[0]
print(f"Integral of 4-term approximation: {ans:.6}")
display_int(taylor8, 0, 1)

(**c**)  Use same approximations to estimate value of $$\int_0^5 \rm{e}^{-x^2} dx$$:

In [ ]:
print("True Result")
int_result(f, 0, 5, showN=True)
for n_terms in [4, 8]:
    g = f.series(x, 0, n_terms).removeO()
    print("%d-term approximation" % (n_terms//2))
    int_result(g, 0, 5, enclose = True, showN=True)

### What?!  Clearly, since the integrand is positive everywhere, a negative integral is *gibberish*.  What is going on here??

Note to reader:  frequently, PChem students just turn these results in, commenting blandly that they are off because it is an approximation....

To explore this further, let's look at a plot of the original function, $\rm{e}^{-x^2}$, along with the various MacLaurin expansions.  (You can save the resulting plot as a pdf to your Downloads file by uncommenting the last line of the following block and running the code block):

In [ ]:
from scipy import vectorize
TG = vectorize(TayGauss)

x_array = linspace(0, 5, 250)
plot(x_array, exp(-x_array**2), lw=4)
for N in [4, 8, 16, 32, 64, 128]:
    # note:  half of terms are zero, hence dividing N by 2 in label
    plot(x_array, TG(x_array, N=N), label = "%d terms" % (N//2))
ylim([-1.5,1.5])
legend()
title("Comparing Gaussian with expansions");
# savefig("expandGauss.pdf") # uncomment to save figure locally to Downloads

It's pretty easy to see visually why the integral of the short expansions give a negative result!